In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### В этом датафрэйме все признаки имеют буквенное обозначение. Таргет обозначени колонкой class

#### Посмотрим на данные

In [5]:
df = pd.read_csv('mushrooms.csv')
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [6]:
df.nunique()

class                        2
cap-shape                    6
cap-surface                  4
cap-color                   10
bruises                      2
odor                         9
gill-attachment              2
gill-spacing                 2
gill-size                    2
gill-color                  12
stalk-shape                  2
stalk-root                   5
stalk-surface-above-ring     4
stalk-surface-below-ring     4
stalk-color-above-ring       9
stalk-color-below-ring       9
veil-type                    1
veil-color                   4
ring-number                  3
ring-type                    5
spore-print-color            9
population                   6
habitat                      7
dtype: int64

In [7]:
# Дизбаланса классов нет, можно использовать accuracy в дальнейшем
df['class'].value_counts()

class
e    4208
p    3916
Name: count, dtype: int64

In [8]:
df.isna().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

#### Обозначим таргет и переменные

In [9]:
X = df.drop('class', axis = 1)

In [10]:
Y = df['class']

In [12]:
replace_dict = {'e': 1, 'p' : 0 }
Y = Y.replace(replace_dict)

C:\Users\KBAKYIIIKA\AppData\Local\Temp\ipykernel_12072\973540507.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y = Y.replace(replace_dict)


In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [14]:
# Закодируем колонки с категориальными признаками и используем модель
catcols = X.select_dtypes(include = ['object']).columns
OHEcols = catcols[X[catcols].nunique() <= 10]
MTEcols = catcols[X[catcols].nunique() > 10]

# Standard Scaler не нужен, так как данные одного порядка
preprocessing = ColumnTransformer([('ohe', OneHotEncoder(), OHEcols),
                                  ('mte', TargetEncoder(), MTEcols)])

# Сравним две модели
model = KNeighborsClassifier(n_neighbors = 5)
model2 = LogisticRegression(penalty = 'l2')

pipe = Pipeline([('preprocessing', preprocessing),
                 ('model', model)
                ])

pipe2 = Pipeline([('preprocessing', preprocessing),
                 ('model', model2)
                ])

In [15]:
OHEcols

Index(['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'stalk-shape',
       'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring',
       'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type',
       'veil-color', 'ring-number', 'ring-type', 'spore-print-color',
       'population', 'habitat'],
      dtype='object')

In [16]:
MTEcols

Index(['gill-color'], dtype='object')

In [25]:
X

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,x,s,n,t,p,f,c,n,k,e,...,s,w,w,p,w,o,p,k,s,u
1,x,s,y,t,a,f,c,b,k,e,...,s,w,w,p,w,o,p,n,n,g
2,b,s,w,t,l,f,c,b,n,e,...,s,w,w,p,w,o,p,n,n,m
3,x,y,w,t,p,f,c,n,n,e,...,s,w,w,p,w,o,p,k,s,u
4,x,s,g,f,n,f,w,b,k,t,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,s,o,o,p,o,o,p,b,c,l
8120,x,s,n,f,n,a,c,b,y,e,...,s,o,o,p,n,o,p,b,v,l
8121,f,s,n,f,n,a,c,b,n,e,...,s,o,o,p,o,o,p,b,c,l
8122,k,y,n,f,y,f,c,n,b,t,...,k,w,w,p,w,o,e,w,v,l


#### Проверим качество

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle = True, random_state = 47)

In [19]:
pipe.fit(X_train, Y_train)
pipe2.fit(X_train, Y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('ohe', OneHotEncoder(),
                                                  Index(['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'stalk-shape',
       'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring',
       'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type',
       'veil-color', 'ring-number', 'ring-type', 'spore-print-color',
       'population', 'habitat'],
      dtype='object')),
                                                 ('mte', TargetEncoder(),
                                                  Index(['gill-color'], dtype='object'))])),
                ('model', LogisticRegression())])

In [20]:
from sklearn.metrics import accuracy_score

In [21]:
print(f'{accuracy_score(pipe.predict(X_test), Y_test)} -- accuracy on test')
print(f'{accuracy_score(pipe.predict(X_train), Y_train)} -- accuracy on train')

1.0 -- accuracy on test
1.0 -- accuracy on train


In [22]:
print(f'{accuracy_score(pipe2.predict(X_test), Y_test)} -- accuracy on test')
print(f'{accuracy_score(pipe2.predict(X_train), Y_train)} -- accuracy on train')

1.0 -- accuracy on test
1.0 -- accuracy on train


#### Качество отличное, откуда можно сделать вывод, что данные распределены линейно и не сложно взаимосвязаны. Отсюда можно сделать вывод, что, опираясь на эту выборку, можно со 100% уверенностью определить класс гриба